In [1]:
import numpy as np
import awkward as ak
import pandas

In [2]:
import sys
sys.path += ["../mlpf/plotting//"]
from plot_utils import pid_to_text, load_eval_data, compute_jet_ratio, compute_met_and_ratio

In [3]:
yvals_qcd, X_qcd, _ = load_eval_data("/local/joosep/mlpf/results/cms/pyg-cms_20240430_094836_751206/preds_checkpoint-22-17.664708/cms_pf_qcd/*.parquet", 1000)

path /local/joosep/mlpf/results/cms/pyg-cms_20240430_094836_751206/preds_checkpoint-22-17.664708/cms_pf_qcd/*.parquet


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [08:49<00:00,  1.89it/s]


In [4]:
met = compute_met_and_ratio(yvals_qcd)

In [5]:
met.keys()

dict_keys(['gen_met', 'pred_met', 'cand_met', 'ratio_pred', 'ratio_cand'])

In [6]:
msk_badmet = np.abs(met["gen_met"] - met["pred_met"])>300

In [7]:
dfs = []
for iev in range(np.sum(msk_badmet)):
    df = pandas.DataFrame()
    X = ak.to_numpy(X_qcd[msk_badmet][iev])
    msk_X = (X[:, 1]>10) | (X[:, 5]>10)
    df["input_type"] = np.array(X[msk_X][:, 0], dtype=np.int32)
    df["input_pt"] = np.array(X[msk_X][:, 1], dtype=np.float32)
    df["input_eta"] = np.array(X[msk_X][:, 2], dtype=np.float32)
    df["input_phi"] = np.arctan2(X[msk_X][:, 3], X[msk_X][:, 4])
    df["input_energy"] = np.array(X[msk_X][:, 5], dtype=np.float32)
    
    df["gen_type"] = np.array(yvals_qcd["gen_cls_id"][msk_badmet][iev][msk_X], dtype=np.int32)
    df["gen_pt"] = np.array(yvals_qcd["gen_pt"][msk_badmet][iev][msk_X], dtype=np.float32)
    df["gen_eta"] = np.array(yvals_qcd["gen_eta"][msk_badmet][iev][msk_X], dtype=np.float32)
    df["gen_phi"] = np.array(yvals_qcd["gen_phi"][msk_badmet][iev][msk_X], dtype=np.float32)
    df["gen_energy"] = np.array(yvals_qcd["gen_energy"][msk_badmet][iev][msk_X], dtype=np.float32)
    
    
    df["pred_type"] = np.array(yvals_qcd["pred_cls_id"][msk_badmet][iev][msk_X], dtype=np.int32)
    df["pred_pt"] = np.array(yvals_qcd["pred_pt"][msk_badmet][iev][msk_X], dtype=np.float32)
    df["pred_eta"] = np.array(yvals_qcd["pred_eta"][msk_badmet][iev][msk_X], dtype=np.float32)
    df["pred_phi"] = np.array(yvals_qcd["pred_phi"][msk_badmet][iev][msk_X], dtype=np.float32)
    df["pred_energy"] = np.array(yvals_qcd["pred_energy"][msk_badmet][iev][msk_X], dtype=np.float32)
    dfs.append(df)

In [8]:
df = pandas.concat(dfs)

In [9]:
df[np.abs(df["gen_pt"] - df["pred_pt"])>200]

,input_type,input_pt,input_eta,input_phi,input_energy,gen_type,gen_pt,gen_eta,gen_phi,gen_energy,pred_type,pred_pt,pred_eta,pred_phi,pred_energy
107,1,539.418396,0.405837,1.778235,584.453552,0,0.000000,0.000000,0.000000,0.000000,1,524.757080,0.403633,1.786849,568.325928
113,5,1537.547485,-0.101370,1.193913,1545.454102,2,1563.101562,-0.102992,1.213735,1571.430298,2,591.884460,-0.094043,1.198891,578.537476
122,6,249.618637,-0.321619,-1.670367,262.640442,1,232.211487,-0.321980,-1.670254,244.352676,0,0.000000,-0.000000,-1.668431,0.000000
131,1,275.898773,-0.321643,-1.670676,290.293640,0,0.000000,0.000000,0.000000,0.000000,1,272.746643,-0.329577,-1.666036,285.000793
16,5,230.397354,1.591700,-0.559053,589.318420,2,343.106445,1.571733,-0.523244,861.839539,2,116.868195,1.677149,-0.566696,312.296814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,6,286.923767,1.021021,-2.842929,449.932953,0,0.000000,0.000000,0.000000,0.000000,1,269.114563,1.050575,-2.841929,422.359009
131,5,581.016113,1.095740,-2.385414,966.138733,2,114.005836,1.109401,-2.401420,191.689545,2,363.666321,1.127951,-2.380907,607.624146
142,6,783.009216,0.511669,2.952585,887.763123,1,589.536133,0.505772,2.951302,666.560608,0,0.000000,0.000000,2.966233,0.000000
147,4,1597.990723,0.494463,2.951312,1797.352783,5,805.447327,0.504974,2.952313,910.349487,5,1167.661133,0.507764,2.950899,1313.734253
